The Augmented Lagrangian
$$
L_{\sigma}(c,b,u;\lambda)=\frac{1}{2}c^{\top}Kc+C\|u_+\|_0+\frac{\sigma}{2}\|u-\big({\mathbf 1}-{\rm diag}(y)K c-b*y-\frac{\boldsymbol{\lambda}}{\sigma}\big)\|^2-\frac{\|\boldsymbol{\lambda}\|^2}{2\sigma}.
$$

In [22]:
# coding=utf-8
#L01 Proximal Operator Prox_{gamma C||(.)_+||}(eta)=argmin_{v} C||(v)_+||_0+1/(2gamma)||v-eta||^2
def Prox(eta,threshold):
    eta[np.where(np.logical_and(eta>0,eta<=threshold))] = 0  #threshold=np.sqrt(2*gamma*C)
    return eta

In [23]:
#Packages
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy import linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_kernels  
import time

Datasets: Breat Cancer, Australian,diabetes, Heart,Fourclass,german.numer

https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/

In [24]:
#Dataset: breast_cancer: 569*30
#https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing 
X, Y = load_breast_cancer(return_X_y=True)
print(X.shape[0])
Y[Y==0]=-1  #Transform label 0，1 to label -1，1

Standardscaler = preprocessing.StandardScaler() 
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
X = Standardscaler.fit_transform(X) 
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.4,random_state=5)
print('X train shape',X_train.shape)
print('X test shape',X_test.shape)
print(y_train[0:10])

569
X train shape (341, 30)
X test shape (228, 30)
[-1 -1  1 -1  1  1  1  1  1  1]


In [25]:
# # #Australian credit approval: 690*14 label=-1,1
# # #https://archive.ics.uci.edu/ml/datasets/Statlog+%28Australian+Credit+Approval%29
# # #https://github.com/wildanalaziz/AustralianCredit/blob/master/australian.csv
# from sklearn import preprocessing 
# import pandas as pd
# df = pd.read_excel('australian.xlsx',header=None)
# df1=np.array(df)
# print(df1.shape)
# X=df1[:,1:15] #d=14
# Y=df1[:,0]
# print(X[0:3,:])

# Standardscaler = preprocessing.StandardScaler() 
# X = Standardscaler.fit_transform(X) 
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.4,random_state=5)
# print(y_train[0:10])

In [26]:
# #diabetes: 768*8, label=-1,1
# #https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/
# from sklearn import preprocessing 
# import pandas as pd
# df = pd.read_excel('diabetes.xlsx',header=None)
# df1=np.array(df)
# print(df1.shape)
# X=df1[:,1:9]   #d=8
# Y=df1[:,0]
# print(X[0:3,:])
# print(Y[0:10])
# Standardscaler = preprocessing.StandardScaler() 
# X = Standardscaler.fit_transform(X) 
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.4,random_state=5)

In [27]:
# #Heart:270*13
# from sklearn import preprocessing 
# import pandas as pd
# df = pd.read_excel('heart.xlsx',header=None)
# df1=np.array(df)
# print(df1.shape)
# X=df1[:,1:14]  #d=13
# Y=df1[:,0]
# Y[Y==0]=-1  #Transform label 0，1 to label -1，1
# Standardscaler = preprocessing.StandardScaler() 
# X = Standardscaler.fit_transform(X) 
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.4,random_state=5)

In [28]:
# #Fourclass:862*2
# from sklearn import preprocessing 
# import pandas as pd
# df = pd.read_excel('fourclass.xlsx',header=None)
# df1=np.array(df)
# print(df1.shape)
# X=df1[:,1:3]  #d=2
# Y=df1[:,0]
# print(X[0:3,:])
# print(Y[0:10])
#Y[Y==0]=-1  #Transform label 0，1 to label -1，1
# Standardscaler = preprocessing.StandardScaler() 
# X = Standardscaler.fit_transform(X) 
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.4,random_state=5)

In [29]:
# #german.numer: 1000*24  label=-1,1
# from sklearn import preprocessing 
# import pandas as pd
# df = pd.read_excel('german.numer.xlsx',header=None)
# df1=np.array(df)
# print(df1.shape)
# X=df1[:,1:25]  #d=24
# Y=df1[:,0]
# print(X[0:3,:])
# print(Y[0:10])
# Standardscaler = preprocessing.StandardScaler() 
# X = Standardscaler.fit_transform(X) 
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.4,random_state=5)

In [30]:
# #svmguide1: 3089*4  label=0,1
# from sklearn import preprocessing 
# import pandas as pd
# df = pd.read_excel('svmguide1.xlsx',header=None)
# df1=np.array(df)
# print(df1.shape)
# X=df1[:,1:5]  #d=4
# Y=df1[:,0]
# print(X[0:5,:])
# Y[Y==0]=-1  #Transform label 0，1 to label -1，1
# Standardscaler = preprocessing.StandardScaler() 
# X = Standardscaler.fit_transform(X) 
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.4,random_state=5)
# print(Y[0:10])

L01KSVMTrain

In [31]:
#L01KSVMTrain
start = time.time()
K = pairwise_kernels(X_train, Y=None, metric='rbf')  # ‘linear’, ‘poly’, ‘rbf’, ‘laplacian
y = y_train
m=np.shape(X_train)[0]

    
#Initialization
c = np.zeros(m)               #coefficients
b=0                           #b=1,-1 bias
u = np.random.random(m)       #loss
Lambda = np.random.random(m)  #Multipliers
max_iter=2000                 #Maximum iteration
beta=np.zeros(4)              #KKT error
tol=1e-3                      #KKT tolerance level 
acc=np.zeros(m)

#Best parameters obtained by Gridseach
C=1
sigma=2
iota=1          #learning_rate=iota*sigma
acc=np.zeros(max_iter)

for i in range(max_iter): 
          
    #updating u
    u_k=np.ones(m)- np.multiply(y,np.dot(K,c))-b*y 
    eta=u_k- Lambda/sigma # eta=np.ones(m)- np.multiply(y,np.dot(K,c))-b*y #np.dot(np.dot(y_diag,K),c)=np.multiply(y,np.dot(K,c))    
    u = Prox(eta,2*C/sigma)   
            
    #updating c   
    xi = np.ones(m)-u- b*y-Lambda/sigma
    c = linalg.solve(K+np.eye(m)/sigma,np.multiply(y,xi))   #np.dot(y_diag,xi)=np.multiply(y,xi)
        
    #updating b
    r = np.ones(m)- u- np.multiply(y,np.dot(K,c))-Lambda/sigma
    b = np.dot(y.T,r)/m
    
    #updating Lambda 
    w = u + np.multiply(y,np.dot(K,c)) + b*y - np.ones(m) 
    index = np.where(u==0)[0]
    index_c= np.nonzero(u)[0]
    Lambda[index]=Lambda[index]+iota*sigma*w[index]      
    Lambda[index_c]=0
    
    
    #Accuracy stopping rule
    acc[i]=accuracy_score(np.sign(-np.dot(K,np.multiply(Lambda,y)) + b),y_train)
    if np.logical_and(i>5,acc[i]>0.9):      #given accuracy tol 0.7
        if np.std(acc[i-2:i+1],ddof=1)<1e-2:  #default 1e-3
            print('Accuracy is stable when i=',i+1)
            break
    
    #proximal stationary point stopping rule
    beta[0]=np.linalg.norm(c+np.multiply(y,Lambda),2)/(1+np.linalg.norm(c,2))#c related to lambda；
    beta[1]=(np.abs(np.dot(y.T,Lambda)))/m  
    beta[2]=np.linalg.norm(w,2)/np.sqrt(m)  
    beta[3]=np.linalg.norm(u_k-u,2)/(1+np.linalg.norm(u,2)) 
    #print('KKT error beta',beta.T)
    if  np.amax(beta)<tol: 
        print('KKT condition (tolerance level=1e-3) is satisfied with iteration i=',i+1)
        break  
        
end = time.time()
print('Program time of L01KSVM is:%ss'%(str(end-start)))
#print(acc.T)

Accuracy is stable when i= 66
Program time of L01KSVM is:0.17353606224060059s


In [32]:
#Output Solutions of L01KSVM
#print('solution b',b)
#print('solution c tranpose',c[np.nonzero(u)[0]].T)
#print('solution u tranpose',u[np.nonzero(u)[0]].T)
#print('solution Lambda:\n',Lambda[np.nonzero(Lambda)[0]].T)
#print('Lambda:\n',Lambda.shape)

In [33]:
#L01KSVM Accuracy 
print('Train Accuracy:\n', accuracy_score(np.sign(-np.dot(K,np.multiply(Lambda,y)) + b),y_train))
K_test = pairwise_kernels(X_test,X_train, metric='rbf') 
print('Test Accuracy:\n', accuracy_score(np.sign(-np.dot(K_test,np.multiply(Lambda,y) ) + b),y_test))

Train Accuracy:
 0.9266862170087976
Test Accuracy:
 0.9473684210526315


Gaussian Kernel SVM with Squared Hinge Loss

In [34]:
#Standard Gaussian Kernel SVM with Squared Hinge Loss
#Best parameter obtained by Gridsearch
start1 = time.time()
#if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
clf=SVC(C=1,kernel='rbf').fit(X_train,y_train)  #default='squared_hinge'
end1 = time.time()
#print('Indices of support vectors for HingeKSVM:\n',clf.support_)
support_vectors =clf.support_vectors_  #Support vectors.
#Decision_function = np.dot(X, clf.coef_[0]) + clf.intercept_[0]
decision_function = clf.decision_function(X_train)
support_vector_indices_canonical = np.where(np.abs(decision_function) <= 1 + 1e-15)[0]
#print('Total number of SV on canonical hyperplane:',support_vector_indices_canonical.shape[0])

In [35]:
#Summary
#L01KSVM 
print({'C':C,'sigma':sigma,'iota':iota})
print('KKT error beta',beta.T)
print('Threshold',np.sqrt((2*C)/sigma))
print('Iteration',i+1)
#dataset
print('Number of Training Data',m)
print('Number of Testing Data',y_test.shape[0])
#Accuracy
print('Train Accuracy of L01KSVM:',accuracy_score(np.sign(-np.dot(K,np.multiply(Lambda,y)) + b),y_train))
print('Test Accuracy of  L01KSVM:',accuracy_score(np.sign(-np.dot(K_test,np.multiply(Lambda,y) ) + b),y_test))
print('Train accuracy of HingeKSVM:',clf.score(X_train,y_train))
print('Test accuracy of HingeKSVM:',clf.score(X_test,y_test))
#Number of Support Vectors
index_sv=np.nonzero(Lambda)[0] 
print('Number of support vectors for L01KSVM',len(index_sv))  
print('Number of support vectors for HingeKSVM:',np.sum(clf.n_support_))
#Indices
print('Indices of support vectors for L01KSVM:\n',index_sv)
#print('Indices of support vectors for HingeKSVM:\n',clf.support_)
#Dual Coefficients
print('Lambda with nonzero elements for L01KSVM:\n',Lambda[np.nonzero(Lambda)])
#print('Dual coefficients of the support vector in the decision function:\n',clf.dual_coef_)
#Computation Time
print('Computation Time for L01SVM',end-start)
print('Computation Time for HingeKSVM',end1-start1)

{'C': 1, 'sigma': 2, 'iota': 1}
KKT error beta [0.07925675 0.00510573 0.12354318 0.17600997]
Threshold 1.0
Iteration 66
Number of Training Data 341
Number of Testing Data 228
Train Accuracy of L01KSVM: 0.9266862170087976
Test Accuracy of  L01KSVM: 0.9473684210526315
Train accuracy of HingeKSVM: 0.9794721407624634
Test accuracy of HingeKSVM: 0.9824561403508771
Number of support vectors for L01KSVM 64
Number of support vectors for HingeKSVM: 90
Indices of support vectors for L01KSVM:
 [  5  10  11  13  18  19  22  23  30  41  51  66  67  68  73  81  84  86
  99 100 102 103 108 111 116 143 149 150 151 153 161 169 170 175 179 189
 193 212 228 233 241 249 250 251 254 255 259 264 266 281 290 292 296 301
 302 306 308 311 313 318 320 322 324 327]
Lambda with nonzero elements for L01KSVM:
 [-1.08447147 -0.2195033  -0.09904906 -0.51651344 -0.43065546 -1.73012995
 -0.25602558 -0.45707928 -1.17546711 -1.13555572 -0.16272257 -0.72170567
 -1.53090878 -0.18735931 -0.41558451 -1.54853074 -0.47675266 -

Numerical Results

In [36]:
#Breast Cancer
# {'C': 16, 'sigma': 2, 'iota': 1}  C=1
# KKT error beta [0.02326828 0.00952901 0.08091771 0.06990977]
# Threshold 4.0
# Iteration 17
# Number of Training Data 341
# Number of Testing Data 228
# Train Accuracy of L01KSVM: 0.9237536656891495
# Test Accuracy of  L01KSVM: 0.9736842105263158
# Train accuracy of HingeKSVM: 0.9912023460410557
# Test accuracy of HingeKSVM: 0.9824561403508771
# Number of support vectors for L01KSVM 79
# Number of support vectors for HingeKSVM: 83
# Indices of support vectors for L01KSVM:
#  [  5  10  13  16  19  20  23  39  40  41  57  66  68  72  80  81  83  84
#   87  98  99 100 102 103 106 108 111 112 116 143 144 150 151 153 154 156
#  161 168 169 172 176 179 183 186 187 189 212 228 233 238 240 241 249 251
#  254 255 259 260 264 273 280 291 292 296 301 302 304 305 308 313 318 320
#  322 323 324 327 330 333 337]
# Lambda with nonzero elements for L01KSVM:
#  [-8.14836890e-01 -3.87560854e-01 -5.75921886e-01 -7.40227821e+00
#  -6.32881302e-01 -3.22445468e+00 -7.68114976e-01 -5.20312408e+00
#  -1.47639054e+00 -1.06918841e+00 -1.89656764e+01 -1.74593159e+01
#  -6.08908156e-01 -2.04389912e-01 -6.88104741e+00 -4.39393185e-01
#  -1.18884160e+01 -3.15353707e+00 -1.56241845e+01 -2.87792460e+00
#  -1.09684552e+00 -3.18539845e-01 -8.56106501e-01 -7.71522466e+00
#  -4.08029130e+00 -1.42429232e+01 -4.07596356e-01 -1.25607243e+01
#  -1.46892294e+01 -2.20069498e+00 -1.51817207e-01 -5.18044473e+00
#  -3.37563349e+00 -4.17678625e+00 -5.29870700e+00 -1.32289959e+01
#  -8.59450210e-01 -4.16162158e+00 -8.19790494e+00 -8.01888185e-01
#  -4.68061374e+00 -7.01548804e-01  8.31811648e-03 -3.05210748e+01
#  -2.30761469e+01 -9.80767559e-01 -4.82677808e+00 -3.24776918e-01
#  -7.94943685e-01 -4.63209044e-02 -2.33766663e+00 -1.28058314e-01
#  -1.17822981e+00 -2.96215676e+00 -1.40079546e+00 -2.32913175e+00
#  -7.41870292e-01 -1.92060817e+00 -5.97978902e-01 -3.55016526e+00
#  -1.90547457e+00 -3.39923412e+00 -7.30600826e-01 -7.65287868e-01
#  -4.40438800e-01 -8.02883044e-01 -2.48700856e+01 -2.79648381e-01
#  -1.14337635e-02 -1.59265782e+00 -2.49062753e+00 -4.31130679e+00
#  -3.17330671e-01 -3.59573928e+00 -4.80319695e+00 -5.61323634e+00
#  -1.06959827e+00 -9.06566794e+00 -2.25517395e+01]
# Computation Time for L01SVM 0.11611390113830566
# Computation Time for HingeKSVM 0.0067157745361328125

In [37]:
#Australian
# {'C': 1, 'sigma': 2, 'iota': 1} C=1
# KKT error beta [0.09495884 0.00294742 0.01371497 0.03939467]
# Threshold 1.0
# Iteration 45
# Number of Training Data 414
# Number of Testing Data 276
# Train Accuracy of L01KSVM: 0.8840579710144928
# Test Accuracy of  L01KSVM: 0.8333333333333334
# Train accuracy of HingeKSVM: 0.9178743961352657
# Test accuracy of HingeKSVM: 0.8405797101449275
# Number of support vectors for L01KSVM 105
# Number of support vectors for HingeKSVM: 188
# Indices of support vectors for L01KSVM:
#  [  0   6  10  12  19  28  35  40  44  48  49  57  62  67  68  90  91  97
#   99 101 102 108 109 112 113 117 118 122 128 136 139 146 150 158 159 167
#  168 173 175 176 180 195 211 212 213 214 216 217 222 224 226 231 233 235
#  240 243 246 247 248 249 251 261 262 263 265 267 270 272 275 277 287 289
#  292 295 296 297 301 302 304 320 326 331 334 336 345 349 350 354 359 360
#  363 378 379 383 384 385 386 388 392 393 394 396 400 404 409]
# Lambda with nonzero elements for L01KSVM:
#  [-0.14745583 -0.50488955 -0.83511497 -0.54799843 -0.23075187 -1.06066406
#  -1.25080577 -1.18332999 -0.47853202 -0.42392439 -0.49168233 -0.13047732
#  -0.23561511 -0.75596544 -0.71698122 -0.17969336 -0.25241137 -0.52031217
#  -0.514475   -1.19823867 -0.22368423 -0.07802037 -0.50245228 -0.74887147
#  -0.81303212 -0.21519037 -0.21672144 -0.24689976 -0.01503003 -0.83090305
#  -0.60696235 -0.51475286 -0.23830915 -0.16925888 -0.39993263 -1.21749845
#  -0.17487259 -0.60976125 -0.84287346 -0.02670318 -0.37815957 -0.60651117
#  -0.49266738 -0.30154815 -1.02077437 -0.47231128 -0.13564935 -1.23125277
#  -0.68825147 -1.44705726 -0.69265961 -0.08250564 -0.24647351 -0.3014923
#  -0.41051123 -0.48155632 -0.28955697 -0.09907649 -0.4008146  -0.48889948
#  -0.3214746  -0.47294679 -0.27233486 -0.04796355 -0.82016883 -0.4750928
#  -1.33337041 -0.43271568 -0.45326341 -0.66668349 -0.41848754 -0.4378123
#  -0.160217   -0.63098137 -0.03547347 -0.22423284 -0.25304009 -0.95285963
#  -1.96980887 -0.00465215 -0.04563515 -0.925455   -0.15159305 -0.20551634
#  -0.64894788 -0.67609247 -0.22631283 -0.0423823  -0.5912854  -0.11434647
#  -0.67214783 -0.47206772 -0.03802587 -0.47762197 -0.23983932 -0.30288442
#  -0.11088944 -0.45958898 -0.44701028 -0.0939639  -0.59030962 -0.0088172
#  -0.14806255 -0.28275703 -0.49309752]
# Computation Time for L01SVM 0.4968292713165283
# Computation Time for HingeKSVM 0.01208639144897461

In [38]:
#Diabetes
# {'C': 1, 'sigma': 2, 'iota': 1}
# KKT error beta [1.24716411e-01 4.57832835e-05 1.55856330e-01 1.29246133e-01]
# Threshold 1.0
# Iteration 191
# Number of Training Data 460
# Number of Testing Data 308
# Train Accuracy of L01KSVM: 0.7913043478260869
# Test Accuracy of  L01KSVM: 0.7824675324675324
# Train accuracy of HingeKSVM: 0.8152173913043478
# Test accuracy of HingeKSVM: 0.7727272727272727
# Number of support vectors for L01KSVM 98
# Number of support vectors for HingeKSVM: 287
# Indices of support vectors for L01KSVM:
#  [  2   6   8  11  17  30  33  34  49  51  66  67  81  82  84  85  87  94
#   99 102 104 107 110 117 124 126 130 131 134 139 149 150 155 165 166 167
#  170 172 191 194 195 199 206 216 219 222 227 233 240 244 246 250 264 269
#  270 276 277 281 283 292 293 294 297 306 309 314 315 319 321 322 324 342
#  346 355 356 358 360 369 370 377 380 383 386 388 397 410 414 417 419 420
#  429 432 435 444 446 447 454 455]
# Lambda with nonzero elements for L01KSVM:
#  [-0.86069979 -0.57701358 -0.92440975 -0.90205203 -0.30038948 -1.02090566
#  -1.24365034 -0.61544926 -0.89533292 -0.18170059 -1.44624878 -1.26503827
#  -0.59198634 -0.7828251  -1.52224594 -1.28949606 -1.10078496 -1.56295067
#  -1.39198632 -0.68962929 -0.742243   -0.49772379 -1.29451985 -1.16385625
#  -0.35308514 -0.86855919 -0.19743434 -1.45591398 -1.07140495 -0.60371941
#  -0.44914752 -2.00949573  0.03588698 -1.13333133 -0.76222419 -0.62377823
#  -0.30122472 -0.51840968 -0.77279714 -0.2105755  -0.25070434 -1.42306534
#  -1.76969305 -1.47565522 -0.26883309 -0.81351647 -0.05145434 -1.47091612
#  -1.36782591 -0.57954527 -1.25027873 -0.59857433 -0.53415048 -1.67256591
#  -0.17595776 -1.72929522 -0.1955009  -0.86079494 -1.52391004 -0.00515805
#  -1.47105061 -1.11355765 -0.59091915 -0.49087456 -0.10205568 -0.77263424
#  -1.82491674 -0.020923   -1.23532977 -0.47481882 -0.76573378 -1.00198496
#  -0.87913464 -0.18394846 -0.65524588 -0.79305367 -1.07575972 -0.47012483
#  -1.18496329 -0.71253922 -1.05552398 -1.02374416 -0.20245625 -1.75458488
#  -1.17739849 -1.43240206 -1.27206982 -0.64764589 -1.31915887 -0.16899236
#  -0.2924013  -0.44796921 -1.69160187 -1.39463239 -1.03843619 -1.64229376
#  -0.02315845 -0.83657417]
# Computation Time for L01SVM 1.6562020778656006
# Computation Time for HingeKSVM 0.007973909378051758

In [39]:
#Heart
# {'C': 1, 'sigma': 1, 'iota': 1} C=1
# KKT error beta [0.12223961 0.00440038 0.35277014 0.45978486]
# Threshold 1.4142135623730951
# Iteration 58
# Number of Training Data 162
# Number of Testing Data 108
# Train Accuracy of L01KSVM: 0.8888888888888888
# Test Accuracy of  L01KSVM: 0.8518518518518519
# Train accuracy of HingeKSVM: 0.9197530864197531
# Test accuracy of HingeKSVM: 0.8425925925925926
# Number of support vectors for L01KSVM 76
# Number of support vectors for HingeKSVM: 97
# Indices of support vectors for L01KSVM:
#  [  0   1   4   7   9  11  14  16  19  20  21  22  24  25  27  28  30  31
#   32  35  36  38  39  42  45  51  57  60  61  64  66  68  69  70  71  73
#   75  79  80  81  83  86  89  95  96  97  98  99 104 106 109 110 111 114
#  116 118 119 120 123 124 125 127 128 129 130 131 134 135 141 142 144 145
#  146 153 155 159]
# Lambda with nonzero elements for L01KSVM:
#  [-1.20993434 -0.4082856  -0.05062558 -0.298124   -1.17137438 -0.96351999
#  -0.85980027 -0.49020929  0.00880023 -0.24447861 -1.38700984 -0.57329918
#  -0.26048563 -0.64184901 -0.7150115  -0.3001303  -0.34359583 -0.29521329
#  -1.15841795 -1.3322757  -1.08208357 -1.45349741 -0.72958231 -1.01631515
#  -0.47218382 -0.65144252 -1.49417254 -1.29988036 -0.82043723  0.06802936
#  -0.47904746 -0.81937592 -0.96002314 -0.13905495 -1.69081409 -1.52188407
#  -0.23503651 -0.78382332 -0.83041955 -1.27436114 -0.58240097 -0.79816747
#  -1.07984589 -1.77358382 -0.27795805 -0.52536517 -1.69562839 -1.39374717
#  -0.51156706 -0.11577442 -0.77752451 -0.65294908 -1.14431733 -1.35001466
#  -0.51334384 -0.57447581 -0.88810162 -0.99231429 -0.17749811 -1.143212
#  -1.4237793  -1.20817712 -0.04940408 -1.61699742 -0.36128413 -0.52893749
#  -1.23689903 -1.32146183 -1.15914041 -0.85636596 -1.01636682 -0.61301109
#  -1.70137148 -1.66383308 -0.97470614 -1.44028552]
# Computation Time for L01SVM 0.06618452072143555
# Computation Time for HingeKSVM 0.0029931068420410156

In [40]:
#fourclass
# {'C': 32, 'sigma': 1, 'iota': 1}  C=2
# KKT error beta [0.00066296 0.00564343 0.10692623 0.03506793]
# Threshold 8.0
# Iteration 301
# Number of Training Data 517
# Number of Testing Data 345
# Train Accuracy of L01KSVM: 1.0
# Test Accuracy of  L01KSVM: 1.0
# Train accuracy of HingeKSVM: 0.9806576402321083
# Test accuracy of HingeKSVM: 0.991304347826087
# Number of support vectors for L01KSVM 85
# Number of support vectors for HingeKSVM: 151
# Indices of support vectors for L01KSVM:
#  [  8  34  42  48  49  53  57  58  69  70  75  78  79  80  87  89  90  91
#   92  97 106 107 110 126 131 132 145 154 164 165 169 172 173 209 215 216
#  221 223 228 231 232 235 241 246 247 249 251 253 259 263 277 280 283 287
#  289 290 301 312 328 333 344 347 364 366 373 383 394 397 403 405 406 424
#  432 443 444 451 453 461 464 467 468 489 501 502 513]
# Lambda with nonzero elements for L01KSVM:
#  [-39.02149694  -9.60526724  -3.29684799 -14.83749206 -35.19022095
#  -24.45180609 -10.08789787 -59.82293619 -16.01066948  -7.25632052
#   -9.46663898  -4.83009331 -17.54949433  -5.75850233 -32.72672877
#  -24.85895068 -39.72317858  -0.98138458  -5.5003142   -0.26250649
#  -18.18836193 -41.11268841  -7.45922072 -49.08414147 -13.89132878
#   -2.25274037  -4.27434593 -18.76450459 -38.01278885  -6.78600866
#  -15.94722394 -55.08960514 -33.59854511 -61.36243098 -42.04610995
#  -56.13829687 -52.29488185 -23.51978554 -44.53178991 -29.02415703
#   -0.44294771 -47.46231556  -9.66139788  -1.72547425 -44.34435144
#  -36.69999164 -62.07159381  -3.11913121 -15.81546647  -6.52265659
#   -3.59365461 -61.00599962 -39.47930608 -30.95362468  -3.94891695
#  -27.22048928 -14.41913167 -60.86204101 -36.32985817  -6.38312207
#   -0.92144751 -44.6003958  -16.85616059  -4.31375149 -12.28081062
#   -0.70609044 -57.99754488 -17.14286859  -1.82425339 -57.35164304
#  -13.91099337 -13.84427669 -53.86139702 -12.55520625 -11.11477448
#   -7.527182   -31.82058071  -3.42234258 -11.90948946 -42.927043
#  -12.27409478 -37.35234113  -2.0576485   -3.15676369  -1.85400292]
# Computation Time for L01SVM 4.291978359222412
# Computation Time for HingeKSVM 0.007918357849121094

In [41]:
#german.numer
# {'C': 2, 'sigma': 2, 'iota': 1}  C=1/2
# KKT error beta [0.11050283 0.00152774 0.29439074 0.35068073]
# Threshold 1.4142135623730951
# Iteration 626
# Number of Training Data 600
# Number of Testing Data 400
# Train Accuracy of L01KSVM: 0.8633333333333333
# Test Accuracy of  L01KSVM: 0.7725
# Train accuracy of HingeKSVM: 0.845
# Test accuracy of HingeKSVM: 0.7725
# Number of support vectors for L01KSVM 284
#Number of support vectors for HingeKSVM: 418
# Indices of support vectors for L01KSVM:
#  [  1   3   8   9  11  12  16  17  18  20  26  28  31  34  35  36  37  38
#   39  41  44  46  49  51  53  55  61  64  66  68  71  73  76  77  84  86
#   89  91  92  93  94  96  97  98 101 102 106 107 112 113 115 116 121 124
#  125 130 131 132 134 139 141 142 143 145 150 153 154 156 157 158 159 160
#  161 162 164 166 168 169 171 172 175 177 180 181 186 188 191 195 196 198
#  201 202 203 206 207 213 215 217 218 224 225 226 228 232 237 238 244 246
#  248 249 250 253 254 256 261 263 265 267 268 269 273 274 275 279 280 281
#  282 284 286 287 288 289 294 296 297 298 303 304 306 309 310 313 314 318
#  321 322 326 327 329 332 333 337 340 341 343 344 345 347 348 350 352 357
#  358 359 360 362 363 364 365 366 368 370 372 373 375 377 382 387 389 391
#  393 395 397 398 399 400 403 404 405 406 407 408 409 411 414 418 419 420
#  422 430 431 432 438 441 442 443 446 450 451 454 455 457 458 462 463 466
#  468 471 473 475 477 479 481 482 483 484 487 488 489 495 496 497 498 499
#  500 502 503 504 506 507 510 511 514 517 521 524 525 526 527 529 532 535
#  538 539 543 546 547 549 550 551 552 555 556 558 559 563 566 567 571 572
#  573 574 575 577 578 579 582 583 586 588 591 594 596 597]
# Lambda with nonzero elements for L01KSVM:
#  [-1.08019222 -0.69273108 -2.25413099 -1.83798402 -1.39283512 -3.25589297
#  -1.58457012 -0.6579276  -2.42650622 -2.44312483 -3.17367941 -0.33883771
#  -1.0522991  -3.07772661 -2.24031929 -1.8701842  -1.69451539 -2.47100057
#  -3.37246552 -0.38402604 -1.29590703 -0.60549042  0.09132691 -2.66189503
#  -1.45609505 -1.22373085 -0.98627152 -1.35455653  0.24637579 -1.73980845
#  -2.11946506 -3.42042153 -0.09145602 -0.23318523 -1.66757943 -2.84190702
#  -1.4877444  -2.06680581 -0.90266445 -1.3781262  -2.63073444 -1.08456823
#  -2.69164318 -2.71606422 -0.45375628 -0.5091855  -2.15489759 -0.52659002
#  -0.70262553 -0.85263301 -3.38663418 -0.10431755 -1.95557453 -0.9964548
#  -2.65844779 -2.76515004 -1.1049988  -1.79468752 -2.15883646 -2.60774564
#  -1.1322652  -0.79980984 -0.59005571 -3.42610381 -2.04047432 -1.54213602
#  -1.65853869 -2.23203133 -1.37537805 -2.29011256 -1.19119386 -0.85602854
#  -3.100912   -3.3889953  -0.89837148 -3.1363182  -1.68560571 -0.8406284
#  -1.26001918 -2.44390779 -1.0751579  -1.78616147 -1.17810562 -1.57960653
#  -0.90322892 -1.03594715 -1.01015745  0.12073448 -1.02988973 -3.3970349
#  -1.48656265 -2.25385631 -3.2000199  -1.23270536 -0.85097438 -3.40054133
#  -0.35188193 -0.67914322 -0.64337994 -1.10328883 -0.69663372 -2.25982197
#  -2.00472253 -0.30670949 -2.15635885 -3.35261375 -2.71946649 -1.8407727
#  -2.62824459 -1.24973508 -0.95354052 -0.38416734 -2.32370767 -1.31440926
#  -0.05320005 -1.05355233 -1.95511134 -0.40112664 -1.17365126 -1.15165095
#  -3.9169565  -0.49395575 -2.03998582 -0.717768   -0.26548576 -2.57813556
#  -2.75786458 -2.95417448 -0.26254825 -0.14608044 -1.5197353  -2.78059371
#  -0.10800177 -1.00427383 -2.06828046 -2.18632758 -0.7475051  -1.69389531
#  -3.03396555 -2.20426607 -1.90531706 -0.68320735 -2.07390143  0.0731663
#  -2.47022798 -1.7042021  -2.72466675 -0.84456865 -2.23533298 -0.78606966
#  -1.73675777 -2.23155609 -1.63503604 -0.52705847 -0.47751857 -3.08410585
#  -0.70542709 -1.24848966 -2.61525423 -2.42639556 -0.44223982 -2.87935891
#  -3.64355263 -0.62508008 -2.04977669 -0.24383496 -0.44094006 -2.27609663
#  -3.45960047 -2.16291966 -1.25636617 -3.19813107 -2.84085637 -1.84289992
#  -1.3123403  -1.33606163 -1.75911345 -1.16031432 -0.73608928 -2.53264577
#  -0.73602079  0.05938741 -1.30007127 -2.4288625  -2.73011043 -0.62808466
#  -3.50243731 -1.51370916 -3.82304965 -3.48300809 -2.5085613  -0.04788765
#  -1.0676226  -3.10164142 -3.74244117 -2.03431577 -0.61074262 -0.17797561
#  -1.33319715 -3.15386954 -0.62080538 -1.69343454 -3.71814576 -0.57796933
#  -2.04497616 -1.0767409  -2.68627095 -2.66889093 -1.47064414 -0.89980649
#  -1.22827226 -0.1129055  -2.4447058  -2.47570118 -0.10198276 -2.24994013
#  -0.80418164 -0.93006408 -0.90588116 -1.9904382  -0.76927855 -0.41019612
#  -3.83933613 -1.1391622  -2.12898808 -0.16724198 -2.17929709 -0.99429361
#  -2.12207012 -0.01961618 -1.60495173 -2.85321519 -3.69499638 -0.71024304
#  -2.66535708 -2.82253562 -3.33375941 -1.36737617 -1.6068685  -2.04241219
#  -0.39866033 -0.94238743 -2.19013141 -1.39817637 -2.95896799 -1.3953755
#  -2.5978621  -1.04156446 -0.82357461 -1.37868671 -2.22434847 -2.81433066
#  -2.18651363 -1.43429255 -1.60754663 -1.84848395 -0.69785811 -2.41998946
#  -0.949533   -0.70736587 -1.58834434 -1.55009588 -1.60867178 -0.26867778
#  -0.75227328 -0.51270641 -0.15991677 -0.72009956 -2.24100644  0.01920046
#  -2.57078394 -1.08692545 -0.5694562  -0.70447613 -1.42016118 -1.2804741
#  -3.19920957 -2.3396192  -2.64026831 -0.22177715 -1.63021361 -2.75992994
#  -3.40848779 -2.02855077]
# Computation Time for L01SVM 10.418253898620605
#Computation Time for HingeKSVM 0.024930715560913086

In [42]:
#svmguide1
# {'C': 1, 'sigma': 2, 'iota': 1}  C=1
# KKT error beta [0.07472558 0.0014923  0.15835737 0.07993511]
# Threshold 1.0
# Iteration 1000
# Number of Training Data 1853
# Number of Testing Data 1236
# Train Accuracy of L01KSVM: 0.9519697787371829
# Test Accuracy of  L01KSVM: 0.9514563106796117
# Train accuracy of HingeKSVM: 0.9724770642201835
# Test accuracy of HingeKSVM: 0.9716828478964401
# Number of support vectors for L01KSVM 139
# Number of support vectors for HingeKSVM: 293
# Indices of support vectors for L01KSVM:
#  [   5   15   38   47   55   75   80  108  112  113  123  125  133  136
#   140  147  149  152  158  181  193  203  226  227  236  248  254  262
#   276  293  323  327  345  389  407  419  426  436  508  513  519  532
#   545  562  570  587  596  598  607  634  650  661  662  684  687  690
#   714  739  743  750  770  780  800  804  811  821  851  873  874  880
#   886  894  900  957  970  994 1016 1022 1023 1047 1054 1063 1069 1102
#  1114 1117 1128 1130 1133 1141 1146 1154 1188 1190 1191 1192 1204 1219
#  1221 1286 1289 1319 1328 1333 1347 1353 1382 1400 1403 1421 1429 1443
#  1470 1472 1482 1484 1504 1515 1518 1525 1556 1599 1605 1621 1624 1632
#  1641 1663 1669 1674 1675 1699 1726 1747 1761 1763 1799 1810 1824]
# Lambda with nonzero elements for L01KSVM:
#  [-1.7306673  -1.32725828 -0.34197006 -0.610644   -0.6293678  -0.23733734
#  -1.08202663 -0.48670301 -0.85079851 -1.39694414 -0.85079851 -0.14694356
#  -0.78232433 -1.19507913 -0.54314496 -1.71071849 -0.13355144 -0.27166199
#  -1.8044749  -1.3902847  -1.60334828 -0.17654337 -1.1743206  -1.35383464
#  -0.49935348 -0.78936215 -0.7681365   0.00488156 -0.51506471 -0.84559712
#  -1.79282638 -1.24616942 -1.02823117 -1.3559301  -1.83430162 -1.17645046
#  -0.65864327 -1.92616783 -1.3527515  -1.87162285 -1.61962499 -1.48434758
#  -0.7048682  -1.69154111 -1.45917261 -0.07990702 -0.39588885 -1.10876997
#  -0.29771966 -1.72377623 -1.63797864 -0.77299517 -1.54759097 -1.12541263
#  -0.53542331 -1.20151511 -1.15325924 -0.23099437 -1.53716379 -1.41995533
#  -0.01745192 -0.97106461 -1.73355766 -0.11212437 -1.41733307 -0.17370109
#  -1.86448323 -0.57895498 -1.78329593 -1.3572675  -0.07337381 -0.29863774
#  -1.58870705 -1.37053329 -0.37169321 -1.33133344 -0.90717296 -0.13519235
#  -0.85240061 -1.74621655 -1.58011684 -0.42025366 -2.02007602 -1.44192525
#  -0.67438563 -0.22535868 -1.38994302 -1.23876462 -1.58455834 -0.56402475
#  -1.6172781  -0.70107504 -0.93856893 -1.46205184 -0.51439344 -1.80688526
#  -1.01150097 -1.08519401 -0.87960783 -0.03992153 -1.70512521 -0.83298419
#  -0.37169321 -1.98054233 -0.97516127 -1.44311002 -1.86223911 -0.85613713
#  -1.27715375 -1.51684955 -0.41477033 -0.03964214 -1.08920608 -1.96476073
#  -1.35166002 -0.4657265  -0.26247673 -0.82149977 -1.05933871 -1.39694414
#  -0.21740294 -1.14121522 -1.47414274 -0.98849075 -1.47985487 -0.10949533
#  -0.71616584 -1.94878109 -1.79853986 -1.78225991 -1.50619346 -0.20700801
#  -1.34913537 -1.73648284 -1.04044099 -0.25984724 -1.36909418 -0.77045114
#  -0.63316987]
# Computation Time for L01SVM 191.95521688461304
# Computation Time for HingeKSVM 0.0279238224029541


# {'C': 1, 'sigma': 2, 'iota': 1}  C=1, acc tol=0.9, std= 1e-2
# KKT error beta [0.06260801 0.00058457 0.1650947  0.08935412]
# Threshold 1.0
# Iteration 485
# Number of Training Data 1853
# Number of Testing Data 1236
# Train Accuracy of L01KSVM: 0.9287641662169455
# Test Accuracy of  L01KSVM: 0.9368932038834952
# Train accuracy of HingeKSVM: 0.9724770642201835
# Test accuracy of HingeKSVM: 0.9716828478964401
# Number of support vectors for L01KSVM 146
# Number of support vectors for HingeKSVM: 293
# Indices of support vectors for L01KSVM:
#  [   5   15   22   23   37   38   55   75   80  101  112  113  123  125
#   133  140  147  149  152  157  158  180  181  193  200  203  226  227
#   236  248  254  262  276  293  295  300  323  345  375  389  407  426
#   431  436  501  504  508  513  519  562  570  587  596  598  607  634
#   650  661  684  687  690  714  739  743  770  780  804  811  816  821
#   851  855  874  886  894  900  920  928  938  957  970 1016 1022 1054
#  1063 1075 1102 1122 1128 1130 1133 1141 1154 1187 1188 1190 1191 1204
#  1214 1218 1221 1286 1287 1289 1318 1319 1328 1347 1353 1382 1383 1391
#  1400 1419 1421 1428 1429 1459 1470 1472 1480 1482 1484 1515 1525 1535
#  1575 1581 1599 1605 1621 1624 1632 1641 1670 1675 1699 1716 1726 1742
#  1761 1802 1810 1816 1824 1838]
# Lambda with nonzero elements for L01KSVM:
#  [-1.32026988 -1.21033684 -1.28802205 -1.1475383  -1.07132889 -0.36835444
#  -0.75520236 -0.2599386  -0.56577728 -1.70510365 -0.59343168 -0.26832619
#  -0.59343168 -0.71723603 -1.65414702 -0.8464686  -1.66630113 -0.08700205
#  -0.28860698 -0.14382311 -1.74063691 -0.96760681 -1.53468849 -1.67866996
#  -1.32403173 -0.22091274 -1.03668967 -0.37941761 -0.43992831 -1.5801599
#  -0.71561572 -0.01376398 -0.53663832 -1.82139445 -1.91842284 -0.3243441
#  -0.63442109 -0.47426913 -1.3763701  -1.58409259 -0.91768545 -0.91424679
#  -0.02609701 -1.90954272 -1.89782483 -1.28569899 -1.9829076  -1.8473512
#  -0.95317772 -1.90258693 -1.92154411 -0.26027364 -1.04880215 -1.3014626
#  -0.49613715 -0.77031949 -1.15666799 -0.79865895 -1.16747837 -0.59523141
#  -0.93080786 -1.10298142 -0.12559904 -0.83083921 -0.28846184 -0.99190973
#  -0.14175864 -1.39645541 -1.4780438  -1.04490395 -1.14507872 -1.43662358
#  -1.32228112 -0.07807181 -0.32203125 -1.53734029 -1.71554936 -1.74354442
#  -1.57574572 -1.68589612 -1.10140129 -0.83405501 -0.19083287 -1.51232611
#  -0.52516981 -0.24517016 -1.40386969 -1.61863386 -1.26024785 -1.11715774
#  -0.68054888 -0.60295157 -1.18728899 -1.02267072 -1.30531838 -1.53873205
#  -1.19263596 -1.10990058 -0.25801535 -1.64719615 -0.10110779 -0.21564305
#  -1.39274361 -0.50494524 -1.46675697 -1.68531933 -1.10140129 -1.06723912
#  -0.18508762 -1.633176   -0.17538698 -1.19099962 -1.12955035 -0.61632102
#  -0.50860821 -0.64620474 -1.58621142 -0.92715198 -1.09999269 -1.25745281
#  -1.69613571 -1.19456812 -1.38227114 -1.14754655 -0.26832619 -0.85329089
#  -0.53683192 -0.3604631  -1.4142618  -1.53032284 -1.3935674  -1.47504247
#  -0.29909276 -1.56581601 -1.55038597 -1.24776699 -1.78642386 -1.56159993
#  -1.23818059 -1.27107868 -0.53235014 -0.82617958 -0.82851652 -0.91945082
#  -0.33830041 -0.50042188]
# Computation Time for L01SVM 94.48719477653503
# Computation Time for HingeKSVM 0.03063035011291504


# {'C': 1, 'sigma': 2, 'iota': 1}
# KKT error beta [0.06817187 0.00125554 0.18050886 0.13769905]
# Threshold 1.0
# Iteration 118
# Number of Training Data 1853
# Number of Testing Data 1236
# Train Accuracy of L01KSVM: 0.9481921208850512
# Test Accuracy of  L01KSVM: 0.9457928802588996
# Train accuracy of HingeKSVM: 0.9724770642201835
# Test accuracy of HingeKSVM: 0.9716828478964401
# Number of support vectors for L01KSVM 171
# Number of support vectors for HingeKSVM: 293
# Indices of support vectors for L01KSVM:
#  [   0    5    7   15   23   37   38   47   75   78   80  101  108  112
#   113  123  125  130  132  133  134  136  140  149  152  157  180  181
#   193  197  200  226  227  236  248  249  254  261  276  293  295  300
#   327  332  340  375  380  384  389  407  426  431  461  468  472  494
#   513  519  532  545  550  562  570  585  587  596  598  607  608  634
#   650  660  661  662  682  684  687  690  714  743  770  773  774  780
#   784  800  804  821  843  851  873  880  886  894  900  928  938  994
#  1016 1022 1023 1044 1047 1061 1063 1069 1075 1102 1117 1118 1128 1130
#  1133 1141 1154 1187 1188 1190 1204 1214 1218 1219 1221 1273 1286 1330
#  1347 1353 1357 1381 1382 1383 1391 1400 1421 1428 1459 1480 1482 1484
#  1503 1510 1515 1518 1525 1528 1535 1536 1575 1599 1602 1605 1621 1624
#  1625 1632 1673 1674 1699 1705 1722 1726 1735 1761 1763 1795 1802 1810
#  1816 1834 1838]
# Lambda with nonzero elements for L01KSVM:
#  [ 0.03307696 -1.60873503 -1.24194545 -1.12299676 -1.26277673 -1.01447902
#  -0.42848808 -1.76745614 -0.69595517 -1.95924406 -0.61624859 -1.04683602
#  -0.37251742 -0.72751752 -1.53268934 -1.05284309 -0.97496569 -0.29588797
#  -0.29704055 -0.74897967 -0.18968203 -1.09369533 -0.79789464 -0.11878899
#  -0.32613628  0.03929192 -0.72661301 -1.28535776 -1.57398816 -1.73729716
#  -1.97555475 -1.27552732 -1.5011795  -0.28613209 -0.76888331 -1.76389969
#  -1.58487324 -1.97747979 -0.60482934 -0.85859891 -0.86608121 -0.90412935
#  -1.54648371 -1.76481297 -1.98091457 -1.84376333 -0.92041076 -1.34560463
#  -1.54731507 -1.66976152 -1.27344501 -0.10743032 -1.76320033 -1.79011406
#  -0.87934825 -0.48629139 -0.44510108 -0.47147469 -1.15352567 -0.83343872
#  -1.78236722 -0.34084562 -1.34375864 -0.03319668 -0.04470227 -1.59084559
#  -0.83706673 -0.4624796  -1.87542    -0.70388978 -1.72965515 -0.37746004
#  -0.7903376  -0.84871855 -0.51596977 -1.22569128 -0.68575319 -1.16211126
#  -1.07805733 -0.96493029 -1.28666311 -0.34210325 -1.13394688 -0.8703596
#  -0.87100373 -0.98855369 -0.08549513 -1.20897802  0.06165755 -0.51153016
#  -0.8298321  -1.24950082 -1.25006332 -0.35925491 -1.63228398 -0.54737864
#  -1.04617619 -1.27431529 -0.74894807 -0.18575331 -1.14525745 -1.33560501
#  -1.97268235 -0.0965659  -1.54366413 -1.08905663 -1.39803056 -0.57274343
#  -1.33599956 -0.8389043  -1.34461024 -0.9831495  -0.69548299 -0.67008042
#  -0.66086254 -0.97764991 -1.26383971 -0.65677001 -1.45168364 -0.79992289
#  -1.39532547 -1.05640359 -1.50365397 -0.2698902  -1.10554449 -0.63963375
#  -1.87064823 -1.59825302 -0.31450334 -1.58180515 -1.93810105 -1.1836158
#  -0.65996627 -1.26390079 -2.01499911 -0.03925183 -1.16150419 -1.28083421
#  -0.76525729 -1.94040698 -0.91416855 -0.93731333 -1.05090404 -0.92392888
#  -1.53268934 -1.24898542 -0.82719452 -0.14615947 -1.43400363 -0.73583612
#  -1.63739577 -1.9707021  -1.66212776 -1.31839178 -1.12376391 -0.55673234
#  -0.12825934 -0.91224302 -0.92494061 -0.17455036 -0.47577623 -1.37605702
#  -0.08477789 -1.2582343  -1.3532161  -0.70013723 -1.22202414 -0.89168208
#  -1.15020685 -0.83851083 -1.15464793]
# Computation Time for L01SVM 22.477457284927368
# Computation Time for HingeKSVM 0.023936748504638672